# Pyspark tutorial 🎯

**Ghi chú:**

 🔹tutorial này được thực hiện trên google colab nên không bị lỗi khi chạy code. Nếu chạy trên jupyter notebook được host bởi local machine chắc chắn sẽ lỗi (chắc vì lý do cấu hình cài đặt hay gì đó :v). Mặc dù đã mò mẫm cả tuần nhưng mình vẫn chưa có giải pháp nào cho vấn đề này nên mình chấp nhận thực hiện nó trên google colab. 🤷‍♂️

 🔹Thêm nữa, tutorial này được xây dựng dựa trên nhiều nguồn, trong đó có các blog mà mình kham khảo. Sau đây xin cảm ơn blog của [longcnttbkhn](https://longcnttbkhn.github.io/huong-dan-spark-co-ban-cho-nguoi-moi/)


![](https://th.bing.com/th/id/OIP.I3eg_GSGbjpQ0O8GDuHVdgHaFL?rs=1&pid=ImgDetMain)

## Installation ⬇️

Để làm việc với spark, chúng ta cần cài đặt các thành phần sau:

- 📌 Java (OpenJDK 8 hoặc 11):

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

- 📌 Apache Spark

  

In [ ]:
# tải gói spark apache về
!wget https://downloads.apache.org/spark/spark-3.5.4/spark-3.5.4-bin-hadoop3.tgz

# giải nén
!tar -xvzf /content/spark-3.5.4-bin-hadoop3.tgz

# di chuyển Spark vào thư mục /opt/spark (đây là quy ước, tự tìm hiểu thêm)
!sudo mv /content/spark-3.5.4-bin-hadoop3 /opt/spark

🔧 Thiết lập biến môi trường như sau:

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/opt/spark"

In [ ]:
# kiểm tra môi trường
! echo $SPARK_HOME
! echo $JAVA_HOME

/opt/spark
/usr/lib/jvm/java-8-openjdk-amd64


- 📌 pyspark

In [ ]:
!pip install pyspark

- 📌 findspark - tùy chọn, nếu chạy trên Google Colab

In [ ]:
!pip install findspark

## Run Spark app 🚀

In [ ]:
# khởi tạo một file .py
! touch firstapp.py

In [ ]:
# ghi nội dung vào file
# mới đầu đừng quan tâm nó code gì, chỉ biết nó chạy là được, biết thì càng tốt :V
%%writefile firstapp.py

from pyspark.sql import SparkSession

if __name__ == "__main__":
    spark = SparkSession.builder \
        .appName("First Spark Application") \
        .master("local[*]") \
        .getOrCreate()

    data = [("Alice", 1), ("Bob", 2)]
    df = spark.createDataFrame(data, ["Name", "Value"])
    df.show()

    spark.stop()

Overwriting firstapp.py


In [ ]:
# kiểm tra nội dung đã ghi vào
! cat firstapp.py


from pyspark.sql import SparkSession

if __name__ == "__main__":
    spark = SparkSession.builder \
        .appName("First Spark Application") \
        .master("local[*]") \
        .getOrCreate()

    data = [("Alice", 1), ("Bob", 2)]
    df = spark.createDataFrame(data, ["Name", "Value"])
    df.show()

    spark.stop()


In [ ]:
# chạy file này
! $SPARK_HOME/bin/spark-submit firstapp.py

25/02/26 09:28:11 INFO SparkContext: Running Spark version 3.5.4
25/02/26 09:28:11 INFO SparkContext: OS info Linux, 6.1.85+, amd64
25/02/26 09:28:11 INFO SparkContext: Java version 1.8.0_442
25/02/26 09:28:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/02/26 09:28:11 INFO ResourceUtils: ==============================================================
25/02/26 09:28:11 INFO ResourceUtils: No custom resources configured for spark.driver.
25/02/26 09:28:11 INFO ResourceUtils: ==============================================================
25/02/26 09:28:11 INFO SparkContext: Submitted application: First Spark Application
25/02/26 09:28:11 INFO ResourceProfile: Default ResourceProfile created, executor resources: Map(cores -> name: cores, amount: 1, script: , vendor: , memory -> name: memory, amount: 1024, script: , vendor: , offHeap -> name: offHeap, amount: 0, script: , vendor: ), task resources: Map(cpus -

Đã có một bảng với 2 cột (Name, Value) cùng với dòng là (Alice, 1) và (Bob, 2). Vậy là ứng dụng đã chạy thành công !

Nếu chạy file .py theo cách bình thường thì mọi tài nguyên và tính toán sẽ được thực hiện trên local machine (trong trường hợp này là colab), nhưng lại bị giới hạn về khả năng xử lý phân tán.

Lệnh `spark-submit` cho phép ta gỡ bỏ điều đó bằng cách "submit"  file .py của bạn đến cluster và  phân phối công việc cho các node trong cluster đó.

Giờ thì ta sẽ đi sâu hơn vào từng phần của mã nguồn.

## SparkSession 🔥

Apache Spark là một hệ thống phân tán, chạy trên nhiều node. Để tương tác với spark, chúng ta cần một thứ gọi là "điểm khởi đầu" (Entry point). Trong trường hợp này chính là SparkSession. SparkSession giống như cửa chính của một tòa nhà, giúp bạn truy cập vào các thành phần bên trong như SparkContext, StreamContext, SQLContext,... (mấy cái này bàn sau).

![](https://abhishekbaranwal10.files.wordpress.com/2018/09/introduction-to-apache-spark-20-12-638.jpg?resize=638%2C479&is-pending-load=1)

In [ ]:
# khởi tạo SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("Datacamp Pyspark Tutorial") \
    .config("spark.memory.offHeap.enabled","true") \
    .config("spark.memory.offHeap.size","5g").getOrCreate()


Dừng SparkSession khi đã hoàn thành tất cả các tác vụ xử lý dữ liệu, nhằm giải phóng tài nguyên (như bộ nhớ, CPU và kết nối đến cluster).

In [ ]:
# spark.stop()

## SparkConf 🔥

SparkConf là một class đặc biệt giúp chúng ta thay đổi lại cách khởi tạo session

Ở phần trên, chúng ta khởi tạo một session thông qua một builder có sẵn và nó khá đơn giản. Nhưng nếu muốn tùy chỉnh cấu hình chi tiết (như bộ nhớ, số lượng core, v.v.) thì ta có cách sau:

In [ ]:
from pyspark import SparkConf


# Tạo SparkConf với các cấu hình cần thiết
conf = SparkConf() \
    .setAppName("MyApp") \
    .setMaster("local[*]") \
    .set("spark.some.config.option", "some-value")

# Tạo SparkSession bằng cách truyền SparkConf
spark = SparkSession \
        .builder \
        .config(conf=conf) \
        .getOrCreate()

# Kiểm tra cấu hình từ SparkSession
print(spark.conf.get("spark.some.config.option"))

some-value


Đối với đa số ứng dụng, việc sử dụng trực tiếp builder của SparkSession đã đủ nên có thể bỏ qua bước này.

## SparkContext 🔥

🔹SparkContext là class cốt lõi trong Apache Spark, giúp ứng dụng kết nối với cluster manager - trình quản lý các tác vụ tính toán phân tán.

🔹SparkContext được sử dụng để khởi tạo các cấu trúc dữ liệu đặc biệt như RDD, accumulator, và broadcast variable (cái này sẽ nói sau)

🔹Trước Spark phiên bản 2.x, SparkContext là entry point chính.

🔹Từ Spark 2.x trở đi, SparkSession thay thế SparkContext, nhưng SparkContext vẫn tồn tại bên trong SparkSession.

Để hiểu sâu hơn, hãy nhìn sơ đồ bên dưới:

![](https://sparkbyexamples.com/wp-content/uploads/2022/05/image04.png)

Như đã nói, khi chạy một Spark app, chương trình sẽ được "submit" tới cluster và được đưa đến các node đang có dữ liệu. Trong đó, một node sẽ được chọn làm **Master node** để chạy một chương trình gọi là **Driver Program**, các node còn lại sẽ đóng vai trò là **Worker**.

Tại **Driver Program**, một đối tượng **SparkContext** sẽ được khởi tạo giúp ứng dụng Spark giao tiếp với **Cluster Manager** để yêu cầu tài nguyên. Sau khi tài nguyên được phân bổ cho các node, **Cluster Manager** sẽ khởi động các **Executor**.

**Executor** là những tiến trình chạy trên các **Worker** và chúng sẽ xử lý các **Task** được giao bởi **Driver Program**.

**Driver Program** sẽ tạo task và phân chia cho các **Worker** theo nguyên tắc xử lý cục bộ, tức là tài nguyên trên node nào sẽ được xử lý bởi **Executor** trên node đó.

Nói tới đây thì có lẽ chúng ta đã định hình được vai trò của SparkContext rồi. Hãy nhớ, bất cứ khi nào làm việc với Spark, luôn khởi tạo SparkContext sau khi đã có SparkSession

In [ ]:
from pyspark import SparkContext

# Khởi tạo sparkContext

try:
  sc = SparkContext("local", "MyApp")
  print(sc)
except:
  print("SparkContext đã tồn tại, chỉ khởi tạo một lần. Đọc thêm ở bên dưới để biết thêm chi tiết")

SparkContext đã tồn tại, chỉ khởi tạo một lần. Đọc thêm ở bên dưới để biết thêm chi tiết


In [ ]:
# kiểm tra có SparkContext nào đang chạy không
SparkContext._active_spark_context

<SparkContext master=local[*] appName=Datacamp Pyspark Tutorial>

🔹Lưu ý: sparkContext chỉ khởi tạo một lần

🔹Chạy lần 2 bị báo lỗi `Cannot run multiple SparkContexts at once; existing SparkContext`

🔹Có thể khắc phục bằng 2 cách:
    🔸Dừng sparkContext,
    🔸Lấy sparkContext hiện có

In [ ]:
# Dừng SparkContext

try:
  # sc.stop()
  # sc = SparkContext("local", "MyApp")
  print(sc)
except:
  print('Lưu ý: Không gọi sc.stop() nếu biến sc chưa tồn tại. Lúc đó phải chạy lệnh bên dưới')

Lưu ý: Không gọi sc.stop() nếu biến sc chưa tồn tại. Lúc đó phải chạy lệnh bên dưới


In [ ]:
# Lấy SparkContext hiện có từ SparkSession

try:
  sc = spark.sparkContext
  print(sc)
except:
  print('Chưa import spark phải không ? nhớ import nha :v')

<SparkContext master=local[*] appName=Datacamp Pyspark Tutorial>


## RDD 🔥

RDD (Resilient Distributed Dataset) là cấu trúc dữ liệu cốt lõi của Apache Spark, cho phép xử lý dữ liệu phân tán trên các cluster một cách linh hoạt và hiệu quả.

![](https://images.viblo.asia/2cd88166-3c16-4cdc-9298-ce9900ac1288.png)

🔴 **Các tính chất của RDD:**

- Tính Phân tán (Distributed): Mỗi RDD được chia thành các phần nhỏ gọi là partitions, mỗi partition có thể được xử lý độc lập trên các node khác nhau trong 1 cluster

- Tính Linh hoạt (Resilient): RDD có thể tự động phục hồi sau khi một phần của dữ liệu hoặc một phần của cluster bị lỗi.

- Tính Bất biến : Sau khi được tạo, một RDD không thể thay đổi.

- Đánh giá lười (Lazy Evaluation): Các phép biến đổi trên RDD không được thực hiện ngay lập tức mà chỉ khi có hành động (action) được gọi.

- Tính tối ưu hóa (Optimized): RDDs có thể tối ưu hóa để tận dụng các hoạt động in-memory, giảm thiểu việc truy cập dữ liệu từ đĩa và tối ưu hóa việc chuyển dữ liệu giữa các phần của RDD trên cluster.

In [ ]:
words = [
    'Scalar',
    'java',
    'hadoop',
    'spark',
    'akka',
    'spark and hadoop',
    'pyspark',
    'pyspark and spark'
]

print(words)
print(type(words))

['Scalar', 'java', 'hadoop', 'spark', 'akka', 'spark and hadoop', 'pyspark', 'pyspark and spark']
<class 'list'>


In [ ]:
rdd_words = sc.parallelize(words)
print(type(rdd_words))

<class 'pyspark.rdd.RDD'>


**🔴 Các hành động (Actions) trên RDD**

Các hành động thực thi tính toán trên RDD và trả về kết quả.
Một số hành động phổ biến:

- collect(): Lấy tất cả phần tử từ RDD về driver.
- count(): Đếm số phần tử trong RDD.
- first(): Lấy phần tử đầu tiên.
- take(n): Lấy n phần tử đầu tiên.
- reduce(f): Gộp các phần tử với một hàm f.

**1️⃣ collect()**

Thu thập toàn bộ dữ liệu từ các phân vùng của RDD trên các worker node và đưa về driver dưới dạng một danh sách Python.

In [ ]:
a = rdd_words.collect()
print(a)
print(type(a))

['Scalar', 'java', 'hadoop', 'spark', 'akka', 'spark and hadoop', 'pyspark', 'pyspark and spark']
<class 'list'>


Khi gọi collect(), Spark sẽ thực hiện toàn bộ các phép biến đổi (transformation) đã được định nghĩa trên RDD để tạo ra dữ liệu cuối cùng.

Lưu ý rằng nó có thể dễ dàng gây ra vấn đề bộ nhớ nếu dữ liệu quá lớn.

**2️⃣count()**

Đếm số lượng phần tử có trong RDD và trả về kết quả dưới dạng một số nguyên (integer).

In [ ]:
print(rdd_words.count())

8


Khi gọi count(), tất cả các phép biến đổi (transformation) đã được định nghĩa trên RDD sẽ được thực hiện để tạo ra dữ liệu cuối cùng trước khi đếm số lượng phần tử.

Không giống như collect, count chỉ trả về số đếm, do đó nó không gây ra quá tải bộ nhớ trên driver ngay cả khi RDD chứa rất nhiều phần tử.

**3️⃣first()**

Chỉ lấy về 1 phần tử mà không cần chuyển toàn bộ dữ liệu về driver, giúp tránh quá tải bộ nhớ.

Nếu RDD rỗng, phương thức này sẽ ném ra lỗi (exception).

In [ ]:
rdd_words.first()

'Scalar'

**4️⃣take()**

`take(n)` trả về một danh sách chứa n phần tử đầu tiên của RDD.

In [ ]:
rdd_words.take(5)

['Scalar', 'java', 'hadoop', 'spark', 'akka']

Khi gọi `take(n)`, Spark sẽ duyệt qua các partition của RDD cho đến khi thu thập đủ n phần tử. Nếu dữ liệu trong RDD ít hơn n, nó sẽ trả về tất cả các phần tử có sẵn.

**🔴Các phép biến đổi (Transformations) trên RDD**

Các phép biến đổi trên RDD tạo ra RDD mới mà không thay đổi RDD ban đầu.

Một số phép biến đổi phổ biến:

- map(f): Áp dụng một hàm f lên từng phần tử.
- filter(f): Lọc các phần tử thỏa mãn điều kiện f.
- flatMap(f): Giống map(), nhưng mở rộng kết quả thành nhiều phần tử.
- groupByKey(): Nhóm các phần tử có cùng khóa (key-value RDD).
- reduceByKey(f): Kết hợp các giá trị có cùng khóa bằng một hàm f.

**filter()**

filter nhận vào một hàm điều kiện (predicate) và áp dụng hàm đó cho từng phần tử của RDD.

Chỉ những phần tử mà hàm trả về True sẽ được giữ lại trong RDD kết quả.

RDD ban đầu không bị thay đổi. Thay vào đó, filter tạo ra một RDD mới chứa các phần tử đã được lọc.

In [1]:
# Tạo RDD từ một danh sách các số từ 1 đến 10
numbers = sc.parallelize(range(1, 11))

# Sử dụng filter để chỉ lấy ra các số chẵn
even_numbers = numbers.filter(lambda x: x % 2 == 0)

# Thu thập kết quả và in ra
print(even_numbers.collect())  # Kết quả: [2, 4, 6, 8, 10]


NameError: name 'sc' is not defined